# Pymaceuticals Inc.
---

### Analysis

- Add your analysis here.
 

In [119]:
%matplotlib notebook
%matplotlib inline

In [120]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single DataFrame
summary_results=pd.merge(study_results,mouse_metadata, on='Mouse ID')

# Display the data table for preview
summary_results=summary_results[['Mouse ID','Timepoint', 'Tumor Volume (mm3)', 'Metastatic Sites',	'Drug Regimen',	'Sex','Age_months',	'Weight (g)']]
summary_results['Tumor Volume (mm3)_display'] = summary_results['Tumor Volume (mm3)'].map('{:.1f}'.format)
# summary_results['Tumor Volume (mm3)']=summary_results['Tumor Volume (mm3)'].round(1)
summary_results.head()
# Display the data table for preview

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g),Tumor Volume (mm3)_display
0,b128,0,45.0,0,Capomulin,Female,9,22,45.0
1,f932,0,45.0,0,Ketapril,Male,15,29,45.0
2,g107,0,45.0,0,Ketapril,Female,2,29,45.0
3,a457,0,45.0,0,Ketapril,Female,11,30,45.0
4,c819,0,45.0,0,Ketapril,Male,21,25,45.0


In [121]:
# Checking the number of mice.
mice_count=summary_results['Mouse ID'].nunique()
mice_count

249

In [122]:
# Our data should be uniquely identified by Mouse ID and Timepoint
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint.
duplicates = summary_results[summary_results.duplicated(subset=['Mouse ID', 'Timepoint'], keep=False)]
duplicate_mouse_ids = duplicates['Mouse ID'].unique()

duplicate_mouse_ids


array(['g989'], dtype=object)

In [123]:
# Optional: Get all the data for the duplicate mouse ID.
duplicates_data = summary_results[summary_results.duplicated(subset=['Mouse ID', 'Timepoint'], keep=False)]

duplicates_data

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g),Tumor Volume (mm3)_display
107,g989,0,45.000000,0,Propriva,Female,21,26,45.0
137,g989,0,45.000000,0,Propriva,Female,21,26,45.0
329,g989,5,48.786801,0,Propriva,Female,21,26,48.8
360,g989,5,47.570392,0,Propriva,Female,21,26,47.6
620,g989,10,51.745156,0,Propriva,Female,21,26,51.7
681,g989,10,49.880528,0,Propriva,Female,21,26,49.9
815,g989,15,51.325852,1,Propriva,Female,21,26,51.3
869,g989,15,53.442020,0,Propriva,Female,21,26,53.4
950,g989,20,55.326122,1,Propriva,Female,21,26,55.3
1111,g989,20,54.657650,1,Propriva,Female,21,26,54.7


In [124]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
summary_results_cleaned = summary_results[~summary_results['Mouse ID'].isin(duplicate_mouse_ids)]

summary_results_cleaned.head()

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g),Tumor Volume (mm3)_display
0,b128,0,45.0,0,Capomulin,Female,9,22,45.0
1,f932,0,45.0,0,Ketapril,Male,15,29,45.0
2,g107,0,45.0,0,Ketapril,Female,2,29,45.0
3,a457,0,45.0,0,Ketapril,Female,11,30,45.0
4,c819,0,45.0,0,Ketapril,Male,21,25,45.0


In [125]:
# Checking the number of mice in the clean DataFrame.
mice_count_cleaned=summary_results_cleaned['Mouse ID'].nunique()
mice_count_cleaned

248

## Summary Statistics

In [127]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen:
# mean, median, variance, standard deviation, and SEM of the tumor volume.
# Assemble the resulting series into a single summary DataFrame.
mean_tumor = summary_results_cleaned.groupby('Drug Regimen')['Tumor Volume (mm3)'].mean()
meadian_tumor =  summary_results_cleaned.groupby('Drug Regimen')['Tumor Volume (mm3)'].median()
var_tumor = summary_results_cleaned.groupby('Drug Regimen')['Tumor Volume (mm3)'].var()
stdev_tumor = summary_results_cleaned.groupby('Drug Regimen')['Tumor Volume (mm3)'].std()
sem_tumor = summary_results_cleaned.groupby('Drug Regimen')['Tumor Volume (mm3)'].sem()
summary_stats = pd.DataFrame({
    'Mean Tumor Volume':mean_tumor,
    'Median Tumor Volume':meadian_tumor,
    'Tumor Volume Variance':var_tumor,
    'Tumor Volume Std. Dev.': stdev_tumor,
    'Tumor Volume Std. Err.':sem_tumor
    })
summary_stats

,Mean Tumor Volume,Median Tumor Volume,Tumor Volume Variance,Tumor Volume Std. Dev.,Tumor Volume Std. Err.
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


In [150]:
# A more advanced method to generate a summary statistics table of mean, median, variance, standard deviation,
# and SEM of the tumor volume for each regimen (only one method is required in the solution)

# Using the aggregation method, produce the same summary statistics in a single line
summary_stats = summary_results_cleaned.groupby('Drug Regimen')['Tumor Volume (mm3)'].agg(['mean', 'median', 'var', 'std', 'sem'])

summary_stats.columns = pd.MultiIndex.from_tuples([('Tumor Volume (mm3)', 'mean'),
                                                   ('Tumor Volume (mm3)', 'median'),
                                                   ('Tumor Volume (mm3)', 'var'),
                                                   ('Tumor Volume (mm3)', 'std'),
                                                   ('Tumor Volume (mm3)', 'sem')])

summary_stats

Tumor Volume (mm3)                                          
                           mean     median        var       std       sem
Drug Regimen                                                             
Capomulin             40.675741  41.557809  24.947764  4.994774  0.329346
Ceftamin              52.591172  51.776157  39.290177  6.268188  0.469821
Infubinol             52.884795  51.820584  43.128684  6.567243  0.492236
Ketapril              55.235638  53.698743  68.553577  8.279709  0.603860
Naftisol              54.331565  52.509285  66.173479  8.134708  0.596466
Placebo               54.033581  52.288934  61.168083  7.821003  0.581331
Propriva              52.320930  50.446266  43.852013  6.622085  0.544332
Ramicane              40.216745  40.673236  23.486704  4.846308  0.320955
Stelasyn              54.233149  52.431737  59.450562  7.710419  0.573111
Zoniferol             53.236507  51.818479  48.533355  6.966589  0.516398

## Bar and Pie Charts

In [130]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using Pandas.


In [131]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using pyplot.


In [132]:
# Generate a pie chart, using Pandas, showing the distribution of unique female versus male mice used in the study

# Get the unique mice with their gender


# Make the pie chart


In [133]:
# Generate a pie chart, using pyplot, showing the distribution of unique female versus male mice used in the study

# Get the unique mice with their gender


# Make the pie chart


## Quartiles, Outliers and Boxplots

In [134]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint


In [135]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers.


    # Locate the rows which contain mice on each drug and get the tumor volumes


    # add subset


    # Determine outliers using upper and lower bounds


In [136]:
# Generate a box plot that shows the distribution of the tumor volume for each treatment group.


## Line and Scatter Plots

In [137]:
# Generate a line plot of tumor volume vs. time point for a single mouse treated with Capomulin


In [138]:
# Generate a scatter plot of mouse weight vs. the average observed tumor volume for the entire Capomulin regimen


## Correlation and Regression

In [139]:
# Calculate the correlation coefficient and a linear regression model
# for mouse weight and average observed tumor volume for the entire Capomulin regimen
